#Librerías

In [26]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install pdfplumber --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 96.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [27]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#Lectura y extracción de la información

In [ ]:
pdf_path = "/content/certificado.pdf"

In [ ]:
 Importar la librería para trabajar con expresiones regulares
import re

header_pattern = r"Cámara de Comercio de Barranquilla\s*CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\s*DE INSCRIPCION DE DOCUMENTOS\.\s*Fecha de expedición:.*?\nRecibo No\..*?\nCODIGO DE VERIFICACIÓN:.*?\n"

In [ ]:
def extract_text_without_header(pdf_path):
      extracted_text = []  # Lista donde almacenaremos el texto de cada página

      with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
          for page in pdf.pages:  # Iterar por cada página del PDF
              text = page.extract_text()  # Extraer texto de la página
              if text:  # Verificar si hay texto en la página
                  text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                  extracted_text.append(text)  # Guardar el texto limpio en la lista

      return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

#NKLT

In [51]:
# Obtener las stopwords predeterminadas de NLTK en español
nltk_stopwords = set(stopwords.words('spanish'))

# Define las stopwords que quiero conservar
stopwords_to_keep = {"sin", "no", "contra"}

# Crear un conjunto final de stopwords a eliminar
final_stopwords = nltk_stopwords - stopwords_to_keep

# --- FUNCIONES DEL PROGRAMA ---

# Limpieza y preparación del texto

def preprocess_text_nltk(text):
    """Preprocesa el texto usando NLTK, conservando números y stopwords específicas."""
    # Normalizar caracteres Unicode y convertir a minúsculas
    text = unicodedata.normalize("NFKD", text).lower()

    # Tokeniza el texto using NLTK, removed language='spanish'
    tokens_nltk = word_tokenize(text)

    # Filtra los tokens: mantiene solo letras alfabéticas y números, y elimina las stopwords menos las stopwords_to_keep
    final_tokens = [
        token for token in tokens_nltk
        if (token.isalpha() or token.isdigit()) and token not in final_stopwords
    ]
    return final_tokens

#
if __name__ == "__main__":
    # 1. Extraer el texto limpio del PDF y eliminar el encabezado
    raw_text = extract_text_without_header(pdf_path)

    if raw_text:
        # 2. Preprocesar el texto con la función de NLTK
        tokens = preprocess_text_nltk(raw_text)

        # 3. Mostrar los primeros 50 tokens como prueba
        print("Primeros 50 tokens preprocesados:")
        print(tokens[:50])

        # 4. Guardar el resultado en un nuevo archivo de texto
        output_file_path = "/content/certificado_limpio.txt"
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(" ".join(tokens))
        print(f"\nTexto preprocesado guardado en: {output_file_path}")
    else:
        print("No se pudo procesar el PDF o no se extrajo texto.")

Primeros 50 tokens preprocesados:
['verifique', 'contenido', 'confiabilidad', 'certificado', 'ingresando', 'digite', 'visualice', 'imagen', 'generada', 'momento', 'puede', 'realizar', 'manera', 'ilimitada', '60', 'calendario', 'contados', 'partir', 'fecha', 'atencion', 'comerciante', 'no', 'cumplido', 'deber', 'legal', 'renovar', 'matricula', 'mercantil', 'd', 's', 'matricula', 'mercantil', 'proporciona', 'seguridad', 'confianza', 'ean', 'n', 'renueve', 'matricula', 'mercantil', 'mas', 'tardar', '31', 'marzo', 'v', 'i', 'c', 'n', 'n', 's']

Texto preprocesado guardado en: /content/certificado_limpio.txt


#SpaCy

In [ ]:
nlp = spacy.load("es_core_news_sm")  #cargar el modelo en español

In [ ]:
  # STOPWORDS
  # NÚMEROS

  # Obtener todas las stopwords predeterminadas de spaCy
  spacy_stopwords = nlp.Defaults.stop_words

  # Define las stopwords que quiero conservar
  stopwords_to_keep = {"sin", "no", "contra"}

  # Crear un conjunto final de stopwords a eliminar
  final_stopwords = spacy_stopwords - stopwords_to_keep

  def preprocess_text(text):
      text = unicodedata.normalize("NFKD", text)
      doc = nlp(text.lower())

      # Se filtran los tokens en aquellos que son solo alfabéticos o solo números y que no están en la lista final de stopwords
      tokens = [token.text for token in doc if (token.is_alpha or token.is_digit ) and token.text not in final_stopwords]
      return tokens

In [ ]:
raw_text = extract_text_without_header(pdf_path)  # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['verifique', 'contenido', 'confiabilidad', 'certificado', 'ingresando', 'digite', 'visualice', 'imagen', 'generada', 'momento', 'ilimitada', 'calendario', 'contados', 'fecha', 'atencion', 'comerciante', 'cumplido', 'deber', 'legal', 'renovar', 'matricula', 'mercantil', 'd', 's', 'matricula', 'mercantil', 'proporciona', 'seguridad', 'confianza', 'laos', 'negocios', 'n', 'renueve', 'matricula', 'mercantil', 'tardar', 'marzo', 'v', 'i', 'c', 'n', 'n', 's', 'fundamento', 'inscripcir', 'ones', 'efectuad', 'as', 'registro', 'mercantil']


In [ ]:
with open("/content/certificado_limpio.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))